### Problem statement
Create a machine learning model to forecast the fuel prices for the service stations in the Rhein-Neckar district based on 
the tanker König dataset.
#### Dataset
The dataset is published on the public website “Tankerkönig“ (http://www.tankerkoenig.de/). This dataset contains the gas prices of all gas stations in Germany from 2014 until today as csv files. A record contains the station id, the datetime, prices for diesel, e5 and e10 and a change indicator. In a separate csv the data of the service stations including its geolocation is provided. 

We use the stations and prices data within this website only for blog posting and demonstration
purpose. For non-commercial use the data is availble with the following license agreement (https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [ ]:
#%pip install hana_ml
#%pip install shapely

In [ ]:
import hana_ml
print(hana_ml.__version__)

In [ ]:
import hana_ml
from hana_ml import dataframe
from hana_ml.dataframe import create_dataframe_from_pandas, create_dataframe_from_shapefile
from hana_ml.algorithms.pal.tsa.additive_model_forecast import AdditiveModelForecast
from hana_ml.algorithms.pal import metrics
print(hana_ml.__version__)

In [ ]:
host = ''
port = 443
user = ''
conn= dataframe.ConnectionContext(address=host, port=port, user=user, 
                               encrypt='true' ,sslValidateCertificate='false')
print(conn.hana_version())


In [ ]:
test_gt_rnk_hdf = conn.table("TEST_GT_RNK_2209", schema="TECHED22")
print(test_gt_rnk_hdf.count())
test_rnk_hdf = conn.table("TEST_RNK_2209", schema="TECHED22")
print(test_rnk_hdf.count())
train_rnk_hdf = conn.table("TRAIN_RNK_2209", schema="TECHED22")
print(train_rnk_hdf.count())

In [ ]:
from hana_ml.algorithms.pal.tsa.additive_model_forecast import AdditiveModelForecast

In [ ]:
conn.sql_tracer.enable_sql_trace(True)
conn.sql_tracer.enable_trace_history(True)

In [ ]:
# create hoildays table, required but kept empty for now
holiday_data_hdf = conn.sql('select * from "TECHED22"."PAL_ADDITIVE_MODEL_ANALYSIS_HOLIDAY"')

In [ ]:
amf = AdditiveModelForecast(massive=True,growth='linear',
                                changepoint_prior_scale=0.06,
                                weekly_seasonality='True',
                                daily_seasonality='True'
                                )

In [ ]:
amf.fit(data=train_rnk_hdf, key="PRICE_AT_TIMESTAMP", group_key="STATION_UUID", holiday=holiday_data_hdf)

In [ ]:
print(conn.last_execute_statement)

In [ ]:
amf.model_.head(5).collect()

In [ ]:
result = amf.predict(data=test_rnk_hdf, key="PRICE_AT_TIMESTAMP", group_key="STATION_UUID")

In [ ]:
# look at predictions for one POS
# predict returns an array of three dataframes. 

result[0].head(10).collect()

### Generate HANA artifacts for CAP project

In [ ]:
from hana_ml.artifacts.generators import hana
from hana_ml.artifacts.generators.hana import HANAGeneratorForCAP

In [ ]:
hanagen = HANAGeneratorForCAP(project_name="PAL_PROPHET_FORECAST_4CAP",
                              output_dir=".",
                              namespace="hana.ml")
hanagen.generate_artifacts(amf, model_position=True, cds_gen=False,tudf=True)